In [66]:
import os
import numpy as np


In [87]:
archivo_csv = 'https://drive.google.com/uc?export=download&id=1a9IfZDuiqbhh2iefH8BMarHL8IXWpayF'
municipios = pd.read_csv(archivo_csv, sep=';', encoding='latin-1')
municipios['Id_Municipio_Dane_text'] = municipios['Id_Municipio_Dane'].astype(str)
#municipios['Id_Municipio_Dane_text']=municipios['Id_Municipio_Dane_text']+'000'


In [91]:
def quitar_cero(texto):
    if texto[0] == '0':
        return texto[1:]
    else:
        return texto

In [106]:
import pandas as pd
# Leer el archivo CSV como DataFrame
df = pd.read_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/seguridad/s_csv/Seguridad_Municipios.CSV', sep=';',encoding='latin-1',dtype={'CODIGO DANE': str} )

C:\Users\ABI\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (3,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [107]:
df['Año'] = df['FECHA HECHO'].str[-4:]


In [108]:
df['temp_ARMAS_MEDIOS']=df['ARMAS MEDIOS']
df['temp_CODIGO DANE']=df['CODIGO DANE']

#df.loc[df['Tipo_delito'] == 'hurto_a_comercio', 'temp_ARMAS_MEDIOS'] = 'X'

df.loc[(df['Tipo_delito'] == 'hurto_a_comercio') & (df['Año'] == '2013'), 'temp_ARMAS_MEDIOS'] = df['CODIGO DANE']
df.loc[(df['Tipo_delito'] == 'hurto_a_comercio') & (df['Año'] == '2013'), 'temp_CODIGO DANE'] = df['ARMAS MEDIOS']

In [109]:
df['temp_CODIGO DANE']=df['temp_CODIGO DANE'].astype(str)
df['Cod_dane'] = df['temp_CODIGO DANE'].apply(lambda x: quitar_cero(x))
#df['Cod_dane'] = df['temp_CODIGO DANE'].apply(lambda x: x[:5])
df['Cod_dane'] = df['Cod_dane'].str.slice(stop=-3)



In [114]:
df = pd.merge(df, municipios, left_on='Cod_dane',right_on='Id_Municipio_Dane_text', how='left')

df = df.dropna(subset=['Id_Municipio_Dane'])
#resultado[resultado['Id_Municipio_Dane'].isna()]


# Categoria de delito por año

In [116]:
seguridad_df = df.loc[:, ['Id_Municipio_Dane', 'Nombre_Municipio','Año','Tipo_delito','CANTIDAD']]
seguridad_df['Cat_seguridad'] = 'Seguridad_' + seguridad_df['Tipo_delito'] + '_' + seguridad_df['Año']
seguridad_df = seguridad_df.loc[:, ['Id_Municipio_Dane', 'Nombre_Municipio','Cat_seguridad','CANTIDAD']]

In [118]:
# Definir las variables para las columnas de agrupación y la columna a sumar
columnas_agrupacion = ['Id_Municipio_Dane', 'Nombre_Municipio','Cat_seguridad']
columna_suma = 'CANTIDAD'

# Agrupar por las columnas de agrupación y sumar la columna a sumar
df_agrupado = seguridad_df.groupby(columnas_agrupacion)[columna_suma].sum()
df_agrupado = df_agrupado.to_frame()

In [120]:
df_pivot = pd.pivot_table(df_agrupado, values='CANTIDAD', index=['Id_Municipio_Dane', 'Nombre_Municipio'],columns='Cat_seguridad', aggfunc=sum,fill_value=0)

In [124]:
# Escribir el DataFrame en un archivo CSV
df_pivot.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/seguridad/s_csv/Seguridad_df.csv')

# Total delitos por año

In [125]:
seguridad_df = df.loc[:, ['Id_Municipio_Dane', 'Nombre_Municipio','Año','CANTIDAD']]
seguridad_df['Cat_seguridad'] = 'Seguridad_' + seguridad_df['Año']
seguridad_df = seguridad_df.loc[:, ['Id_Municipio_Dane', 'Nombre_Municipio','Cat_seguridad','CANTIDAD']]

In [126]:
# Definir las variables para las columnas de agrupación y la columna a sumar
columnas_agrupacion = ['Id_Municipio_Dane', 'Nombre_Municipio','Cat_seguridad']
columna_suma = 'CANTIDAD'

# Agrupar por las columnas de agrupación y sumar la columna a sumar
df_agrupado = seguridad_df.groupby(columnas_agrupacion)[columna_suma].sum()
df_agrupado = df_agrupado.to_frame()

In [127]:
df_pivot = pd.pivot_table(df_agrupado, values='CANTIDAD', index=['Id_Municipio_Dane', 'Nombre_Municipio'],columns='Cat_seguridad', aggfunc=sum,fill_value=0)

In [128]:
# Escribir el DataFrame en un archivo CSV
df_pivot.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/seguridad/s_csv/Seguridad_df_total_año.csv')

In [131]:
df['GENERO'].value_counts()

MASCULINO         2012914
FEMENINO          1517819
NO REPORTA           2556
NO APLICA             542
NO RESPORTADO         372
NO REPORTADO          364
NO REPORTADO           68
-                      63
NO REGISTRA            45
Name: GENERO, dtype: int64

# Genero

In [132]:
seguridad_df = df.loc[:, ['Id_Municipio_Dane', 'Nombre_Municipio','Año','GENERO','CANTIDAD']]
seguridad_df['Cat_seguridad'] = 'Seguridad_' + seguridad_df['GENERO'] + '_' + seguridad_df['Año']
seguridad_df = seguridad_df.loc[:, ['Id_Municipio_Dane', 'Nombre_Municipio','Cat_seguridad','CANTIDAD']]

In [133]:
# Definir las variables para las columnas de agrupación y la columna a sumar
columnas_agrupacion = ['Id_Municipio_Dane', 'Nombre_Municipio','Cat_seguridad']
columna_suma = 'CANTIDAD'

# Agrupar por las columnas de agrupación y sumar la columna a sumar
df_agrupado = seguridad_df.groupby(columnas_agrupacion)[columna_suma].sum()
df_agrupado = df_agrupado.to_frame()

In [134]:
df_pivot = pd.pivot_table(df_agrupado, values='CANTIDAD', index=['Id_Municipio_Dane', 'Nombre_Municipio'],columns='Cat_seguridad', aggfunc=sum,fill_value=0)

In [135]:
# Escribir el DataFrame en un archivo CSV
df_pivot.to_csv('C:/Users/ABI/Documents/1_Maestria/4_Proyect_Final/datos/seguridad/s_csv/Seguridad_df_genero.csv')